In [14]:
import sklearn
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import pickle
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


import joblib

import warnings
warnings.filterwarnings('ignore')

In [15]:
df_raw = pd.read_pickle("./data.pkl")
df_raw

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine,upload_datum,apk
1,2008,169504.0,78,2950.0,0.0,1,18433,18433
2,2007,219718.0,65,1950.0,1.0,1,18433,18433
3,2006,199908.0,95,1895.0,1.0,1,18433,18433
4,2008,138191.0,65,2740.0,1.0,1,18433,18433
5,2006,139057.0,65,2950.0,1.0,1,18433,18433
...,...,...,...,...,...,...,...,...
946,2007,74016.0,78,3199.0,1.0,1,18433,18433
947,2008,138034.0,77,2950.0,1.0,1,18433,18433
949,2007,249820.0,97,1995.0,1.0,1,18433,18433
950,2006,86043.0,78,3950.0,1.0,1,18433,18433


In [16]:
target_names = ["titel", "bouwjaar", "kilometer_stand", "vermogen", "is_handgeschakeld", "is_benzine", "upload_datum", "apk"]

numeric_features = ['bouwjaar', "kilometer_stand", "vermogen", "upload_datum", "apk"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
#         ('text',  CountVectorizer(), 'titel')
    ],
    remainder='passthrough')

In [17]:
df_train, df_test = train_test_split(df_raw, test_size=0.2, random_state=42)

X_train = df_train.drop(['prijs'], axis=1)
y_train = df_train[['prijs']]

X_test = df_test.drop(['prijs'], axis=1)
y_test = df_test[['prijs']]['prijs']
X_test

,bouwjaar,kilometer_stand,vermogen,is_handgeschakeld,is_benzine,upload_datum,apk
735,2006,168147.0,78,1.0,1,18433,18433
799,2011,192186.0,86,1.0,0,18433,18433
572,2007,207118.0,65,1.0,1,18433,18433
392,2011,159031.0,84,1.0,0,18433,18433
386,2017,52947.0,101,1.0,1,18433,18433
...,...,...,...,...,...,...,...
369,2014,120688.0,101,1.0,1,18433,18433
608,2010,169000.0,84,1.0,0,18433,18433
933,2013,152000.0,180,1.0,1,18433,18433
474,2013,119865.0,86,1.0,1,18433,18433


In [18]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [19]:
def evaluate_model(y_test, y_pred):
    print("Mean absolute error:\t", int(mean_absolute_error(y_test, y_pred)))
    print("Mean squared error:\t", int(mean_squared_error(y_test, y_pred)))

In [20]:
dtree = tree.DecisionTreeRegressor()
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)
evaluate_model(y_test, y_pred)

Mean absolute error:	 579
Mean squared error:	 798538


In [21]:
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
evaluate_model(y_test, y_pred)

Mean absolute error:	 505
Mean squared error:	 684579


In [23]:
joblib.dump(random_forest, 'RandomForestRegressor')
joblib.dump(preprocessor, 'RFPreprocessor')

['RFPreprocessor']